# Database Design and Implementation

**Objective**: Our goal was to design a database based on the data collected by the cohort and provide the consolidated code that cleans the data.

# Import of the Libraries

When performing data cleaning, several Python libraries are commonly used to streamline the process.

In [1]:
%pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [2]:
# First we import all the data
import pandas as pd
import numpy as np
import re
from   pandasql import sqldf

# Read of the DataFrame for Future Work

Here, we walk through the process of importing and reading the Excel file into a Python program using the `pandas` library as we saw during the Python class.


In [5]:
# We define the path where the excel is to import it. It is important to adapt
# the path to make the code work
excel_path = "./content/sample_data/All Collected Data.xlsx"

# We read the excel to have it into our program
furniture_df = pd.read_excel(io = excel_path,
                             header = 0,
                             sheet_name = 1,
                             )

# Checking the first five rows
furniture_df.head(5)

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Coffee Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3.0,1.0,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Coffee Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5.0,2.0,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Coffee Tables,Millwood Pines,689.99,169.99,0.0,0.0,0.0,0.0,0.0,3.0,2.0,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Coffee Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3.0,1.0,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Coffee Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9.0,2.0,"Gray; 21.6"" x 20.7"" x 20.7""",0


# Group 1. Review

The "Reviews" column can have two possible states: Standard and Empty Values. In the Standard state, the column contains "No Reviews," and its value will be None or Null. For Empty Values, the column may also contain None or Null, but it could additionally hold a value of 0, depending on whether there is any other review present in the same row.

In [6]:
# Define review columns
review_columns_list = ['Five Star Reviews','Four Star Reviews','Three Star Reviews',
                       'Two Star Reviews','One Star Reviews']

# Count the number of missing (NaN) values in the review columns for each row
# This creates a Series where each entry is the total NaN count for the corresponding row
nan_counts = furniture_df[review_columns_list].isna().sum(axis=1)

# For rows where all review columns are missing (NaN),replace the values in the
# review columns with the None
furniture_df.loc[nan_counts == len(review_columns_list), review_columns_list] = None


# For rows where all review columns are missing (NaN), replace the values in
# the review columns with the 0
furniture_df.loc[nan_counts.between(1, 4), review_columns_list] = furniture_df.loc[
    nan_counts.between(1, 4), review_columns_list].fillna(0)

# We iterate through all the rows
for index, row in furniture_df[review_columns_list].iterrows():
    # We check if all the elements in the row are equal 0
    if sum(row) == 0:
        furniture_df.loc[index, furniture_df[review_columns_list].columns] = None
    # Continuing to the next iteration if any of the data is not equal to 0
    else:
        continue

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Coffee Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3.0,1.0,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Coffee Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5.0,2.0,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Coffee Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3.0,2.0,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Coffee Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3.0,1.0,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Coffee Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9.0,2.0,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,RZM1120,Favorite Characters Fantasy & Sci-Fi Non-Wall ...,Décor & Pillows,Wall Décor,Room Mates,23.99,20.52,46.0,5.0,2.0,2.0,1.0,NaN,NaN,Blue; 33'' H X 42'' W,1
1304,RZM2749,Mickey And Friends Fantasy & Sci-Fi Non-Wall D...,Décor & Pillows,Wall Décor,Room Mates,18.99,16.43,121.0,12.0,5.0,2.0,3.0,NaN,NaN,Blue; 16.75'' H X 39'' W,1
1305,W002143115,Trees & Flowers Non-Wall Damaging,Décor & Pillows,Wall Décor,Ebern Designs,14.99,12.74,1184.0,246.0,78.0,30.0,30.0,NaN,NaN,Teal; 26.75'' H X 33'' W,1
1306,W002302824,Abstract,Décor & Pillows,Wall Décor,George Oliver,26.99,19.99,36.0,14.0,0.0,1.0,3.0,NaN,NaN,"Black, Yellow; 13'' H X 13'' W",1


# Group 2. Size & Color

As the Dual Degree cohort didn't fill in the data for the respective columns, they were retrieved directly from the Wayfair website, using the product code. This was done since neither the quantity of colors nor the quantity of sizes could have changed since the end of Black Friday. Also, if one size or color went out of stock in the meantime, it is still indicated on the website.

Any values as 0 were subtituted for 1 because we assumed that the student misunderstood the assignment, since its impossible to something exist without at least one color and a size.

In [7]:
# Cleaning and validation for "Product Color Count"
furniture_df['Product Color Count'] = furniture_df['Product Color Count'].astype(str).str.strip()

# Convert to numeric
furniture_df.loc[:, 'Product Color Count'] = pd.to_numeric(furniture_df['Product Color Count'], errors='coerce')
furniture_df.loc[furniture_df['Product Color Count'] == 0, 'Product Color Count'] = 1  # Replace 0 with 1
furniture_df = furniture_df[furniture_df['Product Color Count'] > 0]  # Keep values > 0

# Cleaning and validation for "Product Size Count"
furniture_df['Product Size Count'] = furniture_df['Product Size Count'].astype(str).str.strip()

# Convert to numeric
furniture_df.loc[:, 'Product Size Count'] = pd.to_numeric(furniture_df['Product Size Count'], errors='coerce')
furniture_df.loc[furniture_df['Product Size Count'] == 0, 'Product Size Count'] = 1  # Replace 0 with 1
furniture_df = furniture_df[furniture_df['Product Size Count'] > 0]  # Keep values > 0


# Convert back to integer type
furniture_df['Product Color Count'] = furniture_df['Product Color Count'].astype(int)
furniture_df['Product Size Count'] = furniture_df['Product Size Count'].astype(int)

# We show how the dataframe is suffering changes if visible
furniture_df

/var/folders/jz/l65kvw9n2ln9h101ztwmvl980000gn/T/ipykernel_94202/44226284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  furniture_df['Product Size Count'] = furniture_df['Product Size Count'].astype(str).str.strip()


,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Coffee Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Coffee Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Coffee Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Coffee Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Coffee Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware & Bakeware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,Silver; One size,0
1085,EBKN2482,Kitchen Mat,Rugs,Kitchen Mats,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,Pink; One size,0
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Dining Room Sale,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Kitchen & Dining Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0


# Group 3. Subcategories

We cleaned up the Subcategory column to standardize and organize the data, ensuring consistency in classification. The cleaning process involved trimming unnecessary spaces at the beginning and end of each entry, correcting formatting inconsistencies. We carefully addressed potential misclassifications by mapping subcategories to predefined groups based on keywords.

In [8]:
# Preprocess the "Subcategory" column: convert all entries to lowercase and remove extra whitespace
subcategories_list = [str(obj).lower().strip() for obj in furniture_df["Subcategory"]]

# Iterate through each subcategory and its index to modify and classify them into specific categories
for i, subcategory in enumerate(subcategories_list):

    # Check if the subcategory belongs to decor-related items
    if ("accents" in subcategory
        or "decor" in subcategory
        or "décor" in subcategory
        or "art" in subcategory
        or "mirrors" in subcategory
        or "clocks" in subcategory
        or "valances" in subcategory
        or "decals" in subcategory
        or "frames" in subcategory
        or "boards" in subcategory):
        subcategories_list[i] = "Decor"

    # Check for accessories-related items
    elif ("accessories" in subcategory
          or "slipcovers" in subcategory
          or "tableware" in subcategory
          or "drinkware" in subcategory
          or "window" in subcategory):
        subcategories_list[i] = 'Accessories'

    # Check for activity or playroom-related items
    elif ("activity" in subcategory
          or "playroom" in subcategory):
        subcategories_list[i] = 'Activity'

    # Check for greenhouse-related items
    elif "greenhouses" in subcategory:
        subcategories_list[i] = 'Greenhouses'

    # Check for grilling-related items
    elif "grills" in subcategory:
        subcategories_list[i] = 'Grilling'

    # Check for appliances-related items
    elif ("appliances" in subcategory
          or "microwaves" in subcategory
          or "washers" in subcategory
          or "heating" in subcategory
          or "cooktops" in subcategory
          or "fans" in subcategory
          or "fireplaces" in subcategory):
        subcategories_list[i] = 'Appliances'

    # Check for plants or trees-related items
    elif ("plants" in subcategory
          or "trees" in subcategory):
        subcategories_list[i] = 'Plants'

    # Check for renovation-related items
    elif ("remodel" in subcategory
          or "hooks" in subcategory
          or "renovation" in subcategory
          or "flooring" in subcategory
          or "hardware" in subcategory
          or "tiles" in subcategory
          or "plumbing" in subcategory
          or "building" in subcategory):
        subcategories_list[i] = 'Renovation'

    # Check for bedding-related items
    elif ("bedding" in subcategory
          or "pillows" in subcategory
          or "blankets" in subcategory
          or "mattresses" in subcategory):
        subcategories_list[i] = 'Bedding'

    # Check for dog-related items
    elif ("feeding" in subcategory
          or "bowls" in subcategory
          or "dog" in subcategory
          or "fences" in subcategory
          or "grooming" in subcategory):
        subcategories_list[i] = 'Dog'

    # Check for bird-related items
    elif "bird" in subcategory:
        subcategories_list[i] = "Bird"

    # Check for reptile-related items
    elif "reptile" in subcategory:
        subcategories_list[i] = "Reptile"

    # Check for chicken-related items
    elif "chicken" in subcategory:
        subcategories_list[i] = 'Chicken'

    # Check for cat-related items
    elif ("cats" in subcategory
          or "litter" in subcategory
          or "cat" in subcategory):
        subcategories_list[i] = "Cat"

    # Check for cookware-related items
    elif ("cookware" in subcategory
          or "sideboards" in subcategory
          or "bakeware" in subcategory
          or "utensils" in subcategory):
        subcategories_list[i] = 'Cookware'

    # Check for chairs or seating-related items
    elif ("chairs" in subcategory
          or "seating" in subcategory
          or "stools" in subcategory):
        subcategories_list[i] = 'Chairs'

    # Check for tables or tabletop-related items
    elif ("tables" in subcategory
          or "table" in subcategory
          or "tabletop" in subcategory):
        subcategories_list[i] = 'Tables'

    # Check for furniture-related items
    elif ("furniture" in subcategory
          or "bookcases" in subcategory
          or "ottomans" in subcategory
          or "desks" in subcategory
          or "entertainment" in subcategory
          or "space" in subcategory
          or "bookends" in subcategory
          or "cribs" in subcategory
          or "set" in subcategory
          or "sets" in subcategory
          or "stands" in subcategory):
        subcategories_list[i] = 'Furniture'

    # Check for lighting-related items
    elif ("lighting" in subcategory
          or "lamps" in subcategory
          or "lamp" in subcategory
          or "sconces" in subcategory
          or "light" in subcategory
          or "lights" in subcategory
          or "chandeliers" in subcategory):
        subcategories_list[i] = 'Lighting'

    # Check for rugs or mat-related items
    elif ("doormats" in subcategory
          or "rug" in subcategory
          or "rugs" in subcategory
          or "mat" in subcategory
          or "mats" in subcategory
          or "runners" in subcategory
          or "pads" in subcategory):
        subcategories_list[i] = 'Rugs'

    # Check for cabinet-related items
    elif "cabinet" in subcategory:
        subcategories_list[i] = "Cabinet"

    # Check for storage or organization-related items
    elif ("storage" in subcategory
          or "organization" in subcategory
          or "organizer" in subcategory
          or "holders" in subcategory
          or "shelves" in subcategory
          or "cubbies" in subcategory):
        subcategories_list[i] = "Organization"

    # Default case: assign items not matching any condition to "Accessories"
    else:
        subcategories_list[i] = "Accessories"

# Update the "Subcategory" column in the dataframe with the modified classifications
furniture_df["Subcategory"] = subcategories_list

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,Silver; One size,0
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,Pink; One size,0
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0


# Group 4. Product Name

Cleanup the Product Name to get rid of empty spaces at the beginning and the end of names, we also got rid of double quotes “” “” from copying errors, with the condition to only remove the double quotes where they appear at the beginning and end so as to not get rid important name information, like in the case where product name ends in inches, denoted by “ or instances where the double quotes are part of the actual name of the product

In [9]:
# Step 1: Clean the 'Product Name' column
furniture_df['Product Name'] = furniture_df['Product Name'].str.replace(r'^"(.*)"$', r'\1', regex=True)

# Step 2: Remove spaces at the beginning and end of each name
furniture_df['Product Name'] = furniture_df['Product Name'].str.strip()

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,Silver; One size,0
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,Pink; One size,0
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0


# Group 5. Brand

To clean and standardize the Brand column, we begin by fixing any Null values, replacing them with "Unknown." Next, we check the data type of each cell in the column to ensure consistency. We then capitalize the first letter of each word in the brand names. Following this, we remove trademarks such as ™ and ®. Any spaces appearing at the beginning or end of a brand name are trimmed, and extra spaces between characters are eliminated. We manually clean any specific brand names that require individual attention. Finally, we standardize the format for terms like "Co.," "LLC," and "Inc." to ensure uniformity across the dataset.

In [10]:
# 1. Fix the Null Value
furniture_df['Brand']=furniture_df['Brand'].fillna('Unknown')

# 2. Looping over data types for brand
for brand in furniture_df['Brand']:
    if type(brand) != str:
        print(type(brand))

# 3. Handling Capitalization Issues:
furniture_df['Brand'] = furniture_df['Brand'].str.lower().str.title()

# 4. Handling Trademark Symbols:
furniture_df['Brand'] = furniture_df['Brand'].str.replace('™', '', regex=False)
furniture_df['Brand'] = furniture_df['Brand'].str.replace('®', '', regex=False)

# 5. Handling Spacing Issues:
furniture_df['Brand'] = furniture_df['Brand'].str.strip()

# Remove extra spaces between words* (normalize spaces):
furniture_df['Brand'] = furniture_df['Brand'].str.replace(r'\s+', ' ', regex=True)

# 6. Brand Name that need to be cleaned individually
furniture_df['Brand'] = furniture_df['Brand'].str.replace(r'(?i)viv\+rae', 'Viv + Rae', regex=True)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Willa Arlo Interior': 'Willa Arlo Interiors'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Reblilliant': 'Rebrilliant'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Mount Alpi': 'Mont Alpi'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Dovecave': 'Dovecove'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Trix': 'Trinx'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Bath Décor': 'Bath Decor'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Haute Décor': 'Haute Decor'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Ih Casadécor': 'Ih Casadecor'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Caroline\'S Treasures': 'Caroline\'s Treasures'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Gracia Oaks': 'Gracie Oaks'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Gracie Oaks Tamez': 'Gracie Oaks'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Three Post': 'Three Posts'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Wildone Home': 'Wildon Home'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Orean Ellis': 'Orren Ellis'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Wac Lightining': 'Wac Lighting'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'TRUE': 'True'}, regex=False)
furniture_df['Brand'] = furniture_df['Brand'].replace({'Winston Forge': 'Williston Forge'}, regex=False)

# 7. General Pattern Replacement (Handling Specific Patterns or Errors):
furniture_df['Brand'] = furniture_df['Brand'].str.replace(r'\binc\b$', 'Inc.', case=False, regex=True)  # Replace "inc" with "inc." only at the end
furniture_df['Brand'] = furniture_df['Brand'].str.replace(r'\bllc\b$', 'LLC', case=False, regex=True)  # Replace "llc" with "LLC" only at the end
furniture_df['Brand'] = furniture_df['Brand'].str.replace(r'\bco\b$', 'Co.', case=False, regex=True)  # Replace "co" with "co." only at the end

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,Silver; One size,0
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,Pink; One size,0
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0


# Group 6. Original Prize

Basically the cleanup here consisted on making the decimals of the price only two values, and then transforming the type of column as "float".

In [11]:
# Iterate through each row in the DataFrame
for index, row in furniture_df.iterrows():
    if pd.isna(row['Original Price']):
        # If Original Price is blank (NaN), fill it with Discounted Price
        furniture_df.at[index, 'Original Price'] = row['Discounted Price']
    # If it's not blank, we do nothing (move to the next row)
    else:
        pass

# Format the 'Original Price' column values to two decimal places as strings (e.g., "100.00")
furniture_df['Original Price'] = furniture_df['Original Price'].apply(lambda x: f"{x:.2f}")

# Convert the 'Original Price' column values back to floats for further numerical operations
furniture_df['Original Price'] = furniture_df['Original Price'].astype(float)

# Reapply the formatting to two decimal places as strings but handle missing values (NaN) appropriately by leaving them unchanged
furniture_df['Original Price'] = furniture_df['Original Price'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else x)

# Convert to numeric and round to 2 decimal places
furniture_df['Original Price'] = pd.to_numeric(furniture_df['Original Price'], errors='coerce').round(2)

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Description,Black Friday Deal
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,Rustic Brown/White; One size,0
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,"Espresso; 18.2"" H x 41.7"" W x 19.7"" D",0
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,"White; 14"" H x 25"" L x 21.60"" W",0
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,Natural Unfinished; One size,0
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,"Gray; 21.6"" x 20.7"" x 20.7""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,Silver; One size,0
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,Pink; One size,0
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,Blue; One size,0


# Group 7. Description

In the process of data cleaning, several key assumptions are made to ensure consistency and clarity in the output. Firstly, it is assumed that during data collection, colors are recorded before sizes, maintaining this order throughout the dataset. Cleaned data will be standardized to include exactly one color and one size for each entry. In cases where either a color or size is missing, the output will default to "One color" and "One size," as previously agreed upon. For entries containing multiple colors or sizes, the first recorded color and the last recorded size will be selected to represent the data. These assumptions streamline the cleaning process and provide a clear structure for the dataset.

In [12]:
# Initialize empty lists to store extracted color and size information
color = []
size = []

# Loop through each item in the "Description" column of the DataFrame
for descr in furniture_df["Description"]:

    descr_str = str(descr)  # Ensure the description is treated as a string

    # If the description contains a semicolon (';'), split the string using it
    if ';' in descr_str:
        splited = descr_str.split(";")

        # Assume the first part corresponds to color
        color.append(splited[0])

        # Assume the last part corresponds to size
        size.append(splited[len(splited) - 1])

    # If the description contains a colon (':'), split the string using it
    elif ':' in descr_str:
        splited = descr_str.split(":")

        # Assume the first part corresponds to color
        color.append(splited[0])

        # Assume the last part corresponds to size
        size.append(splited[len(splited) - 1])

    # If the description contains a comma (','), split the string using it
    elif ',' in descr_str:
        splited = descr_str.split(",")

        # Assume the first part corresponds to color
        color.append(splited[0])

        # Assume the last part corresponds to size
        size.append(splited[len(splited) - 1])

    # If none of the above delimiters are found, handle as missing color
    else:
        # Default color is "One Color" and size is the entire description
        color.append("One Color")
        size.append(descr_str)

# Standardize Color Information
color_filtered = []

for c in color:
    # If the color is "N/A", standardize it to "One Color"
    if "N/A" in c:
        color_filtered.append("One Color")
    else:
        # If color is missing, default, or invalid, set it to "One Color"
        if len(c) == 0 or "default" in c.lower():
            color_filtered.append("One Color")
        else:
            # Split color values using '/' or ',' as delimiters
            splited = re.split(r'[/,]+', c)

            # Clean up each color: remove line breaks, trim spaces, and capitalize words
            color_filtered.append(splited[0].replace("\n", "").strip().title())

# Standardize Size Information
size_filtered = []

for s in size:
    # If the size is "N/A", standardize it to "One Size"
    if "N/A" in s:
        size_filtered.append("One Size")
    else:
        # If size is missing, default, or invalid, set it to "One Size"
        if len(s) == 0 or "default" in s.lower() or "onesize" in s.lower():
            size_filtered.append("One Size")
        else:
            # Split size values using '/' or ',' as delimiters
            splited = re.split(r'[/,]+', s)

            # Clean up each size: remove line breaks, trim spaces, and capitalize words
            size_filtered.append(splited[0].replace("\n", "").strip().title())

# Remove the original "Description" column from the DataFrame
furniture_df = furniture_df.drop(columns=["Description"])

# Add the standardized color and size information as new columns in the DataFrame
furniture_df["Color"] = color_filtered
furniture_df["Size"] = size_filtered

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Black Friday Deal,Color,Size
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,0,Rustic Brown,One Size
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,0,Espresso,"18.2"" H X 41.7"" W X 19.7"" D"
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,0,White,"14"" H X 25"" L X 21.60"" W"
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,0,Natural Unfinished,One Size
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,0,Gray,"21.6"" X 20.7"" X 20.7"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,0,Silver,One Size
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,0,Pink,One Size
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,0,Blue,One Size
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.9,NaN,NaN,NaN,NaN,NaN,1,1,0,Blue,One Size


# Group 8. Discounted Price

We changed values in Discounted Price that could not be converted to float because it had symbols such as “-“ instead of “.”
We added two decimal places to all values and converted everything to float.
We flipped the values that were wrongly added to Original Price and Discount Price. We flipped the rows in which Discounted Price is higher than Original Price.

In [13]:
# Replace dashes with dots in the "Discounted Price" column
furniture_df["Discounted Price"] = furniture_df["Discounted Price"].apply(
    lambda value: value.replace("-", ".")
    if isinstance(value, str) and "-" in value
    else value
)

# Format "Discounted Price" values to two decimal places as strings
furniture_df["Discounted Price"] = furniture_df["Discounted Price"].astype(float).map("{:.2f}".format)

# Convert "Discounted Price" back to float for further numerical operations
furniture_df["Discounted Price"] = furniture_df["Discounted Price"].astype(float)

# Loop through each row of the DataFrame to validate pricing logic
for index, row in furniture_df.iterrows():
    # Check if "Discounted Price" is greater than "Original Price"
    if row["Discounted Price"] > row["Original Price"]:
        # Swap the values of "Discounted Price" and "Original Price" to maintain logical consistency
        furniture_df.at[index, 'Discounted Price'], furniture_df.at[index, 'Original Price'] = \
        row['Original Price'], row['Discounted Price']
    else:
        # If no issue is found, do nothing
        pass

# We show how the dataframe is suffering changes if visible
furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,Two Star Reviews,One Star Reviews,Product Color Count,Product Size Count,Black Friday Deal,Color,Size
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,2.0,1.0,3,1,0,Rustic Brown,One Size
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,79.0,113.0,5,2,0,Espresso,"18.2"" H X 41.7"" W X 19.7"" D"
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,NaN,NaN,3,2,0,White,"14"" H X 25"" L X 21.60"" W"
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,6.0,3.0,3,1,0,Natural Unfinished,One Size
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,0.0,0.0,9,2,0,Gray,"21.6"" X 20.7"" X 20.7"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Kitchen & Tabletop,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,0.0,0.0,1,1,0,Silver,One Size
1085,EBKN2482,Kitchen Mat,Rugs,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,0.0,0.0,1,1,0,Pink,One Size
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Dining Room,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,NaN,NaN,1,1,0,Blue,One Size
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Furniture,Furniture,Everly Quinn,5299.90,3899.90,NaN,NaN,NaN,NaN,NaN,1,1,0,Blue,One Size


# Group 9. Category

We brought down the categories from 51 to 11.
We included the 9 categories, which were the rooms assigned to the teams. 2 additional categories (rugs and lighting) were created as it was not possible to include them in the original 9 rooms.

In [15]:
# We create two empty lists to store the cleaned categories and subcategories
categories_list = []
subcategories_list = []

# Save all the categories from the DataFrame column into the list as lowercase strings
for obj in furniture_df["Category"]:
    categories_list.append(str(obj).lower())

# Save all the subcategories from the DataFrame column into the list as lowercase strings
for obj in furniture_df["Subcategory"]:
    subcategories_list.append(str(obj).lower())

# Iterate through each category and its index to perform modifications
for i, category in enumerate(categories_list):
    # Check if the category contains specific keywords for kitchen-related categories
    if ("appliances" in categories_list[i]
        or "dinning" in categories_list[i]
        or "kithcen" in categories_list[i]
        or "improvement" in categories_list[i]):
       categories_list[i] = "Kitchen & Dining"

    # If the category is "baby & kids", change it to "Baby & Kids"
    elif category == "baby & kids":
       categories_list[i] = "Baby & Kids"

    # If the category is "bathroom", change it to "Bathroom"
    elif category == "bathroom":
       categories_list[i] = "Bathroom"

    # Handle "bed & bath" by checking subcategories to determine whether it's "Bedroom" or "Bathroom"
    elif category == "bed & bath":
        if "bed" in subcategories_list[i] or "mattress" in subcategories_list[i]:
            categories_list[i] = "Bedroom"
        else:
            categories_list[i] = "Bathroom"

    # If the category is already "bedroom", ensure it is labeled as "Bedroom"
    elif category == "bedroom":
        categories_list[i] = "Bedroom"

    # Check if subcategories contain any pet-related keywords, and change the category to "Pet"
    elif ("bird" in subcategories_list[i]
        or "cat" in subcategories_list[i]
        or "chicken" in subcategories_list[i]
        or "dog" in subcategories_list[i]
        or "fish" in subcategories_list[i]
        or "pets" in subcategories_list[i]
        or "reptile" in subcategories_list[i]
        or "small animal" in subcategories_list[i]):
        categories_list[i] = "Pet"

    # Check if categories contain outdoor-related keywords, and change to "Outdoor & Patio"
    elif ("outdoor" in categories_list[i]
        or "doormats" in categories_list[i]
        or "renovation" in categories_list[i]):
        categories_list[i] = "Outdoor & Patio"

    # Check for living room-related keywords and change the category to "Living Room"
    elif ("liv" in categories_list[i]
        or "pillows" in categories_list[i]):
        categories_list[i] = "Living Room"

    # Check for entry-related keywords and change the category to "Entry & Mudroom"
    elif ("hallway" in categories_list[i]
        or "shoe" in categories_list[i]
        or "entry" in categories_list[i]):
        categories_list[i] = "Entry & Mudroom"

    # If the category is "lighting", change it to "Lighting"
    elif category == "lighting":
       categories_list[i] = "Lighting"

    # Check for office-related keywords and change the category to "Office"
    elif ("office" in categories_list[i]
        or "organi" in categories_list[i]):
        categories_list[i] = "Office"

    # If the category is "rugs", change it to "Rug"
    elif category == "rugs":
       categories_list[i] = "Rug"

    # Handle "furniture" by examining subcategories for further classification
    elif category == "furniture":
        if "kitchen" in subcategories_list[i] or "dining" in subcategories_list[i]:
            categories_list[i] = "Kitchen & Dining"
        elif "living" in subcategories_list[i]:
            categories_list[i] = "Living Room"
        elif "mud" in subcategories_list[i]:
            categories_list[i] = "Entry & Mudroom"
        elif "bed" in subcategories_list[i]:
            categories_list[i] = "Bedroom"
        elif "bath" in subcategories_list[i]:
            categories_list[i] = "Bathroom"
        else:
            categories_list[i] = "Office"

    # Default case for categories not matching any specific conditions
    else:
        if "kitchen" in subcategories_list[i]:
            categories_list[i] = "Kitchen & Dining"
        else:
            categories_list[i] = "Lighting"

# Add the modified categories list as a new "Category" column
furniture_df["Category"] = categories_list

# Save the cleaned DataFrame to a new CSV file
furniture_df.to_csv("data_cleaned_all.csv", index=False)

DD Team 1

In [34]:
# DD Team 1

# Eliminate an unnecessary space and remove unwanted characters
furniture_df['Converted_Size'] = (
    furniture_df['Size']
    .str.lower()
    .str.replace("''", '', regex=False)  # Remove single quates
    .str.replace('"', '', regex=False)  # Remove double quotes
    .str.replace(' ', '', regex=False)  # Remove spaces
     )

# Function to convert feet and inches to inches and retain relevant parts
def convert_feet_to_inches(element):
    # Split the element into parts using delimiters like 'x', 'w', 'h', 'd'
    parts = re.split(r'(?<=[xwxdhl])', element)  # Splits dimensions and keeps suffixes
    converted_parts = []

    for part in parts:
        # Find all matches of feet and inches in the part
        matches = re.findall(r'(\d+)\'(\d+)?|(\d+)\'', part)
        if matches:
            for match in matches:
                # Handle patterns like 7'6, 10', etc.
                # Feet from group 1 (like 7 of 7'6) or group 3 (like 10 of 10')
                if match[0]:
                    feet = int(match[0])
                else:
                    feet = int(match[2])
                # Inches from group 2 (like 6 of 7'6)
                if match[1]:
                    inches = int(match[1])
                else:
                    inches = 0
                # Calculate total inches
                total_inches = feet * 12 + inches
                # Replace with inches
                part = re.sub(r'(\d+)\'(\d+)?|(\d+)\'', str(total_inches), part, 1)
        converted_parts.append(part.strip())

    # Join the converted parts back into a string
    result = ''.join(converted_parts)
    return result

# Apply the conversion function to the Size column
furniture_df['Converted_Size'] = furniture_df['Converted_Size'].astype(str)
furniture_df['Converted_Size'] = furniture_df['Converted_Size'].apply(convert_feet_to_inches)

# Adjust the format to extract height, width, and depth values,
# assuming **h x ** w x ** d is the right format
# dimension of 'l' is converted to other dimensions, prioritizing with the order of h > w > d

# Function to covert the fromat to extract dimension numbers

def convert_size_format(element):
    # Extract numbers
    def extract_number(part):
        match = re.search(r'(\d+(\.\d+)?)', part)
        if match:
            return match.group(1)
        else:
            return ''

    # num -> num + h
    if re.fullmatch(r'\d+', element):
        return f"{element}h"

    # num x num -> num h x num w
    elif re.fullmatch(r'\d+(\.\d+)?x\d+(\.\d+)?', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}w"

    # num x num x num -> num h x num w x num d
    elif re.fullmatch(r'\d+(\.\d+)?x\d+(\.\d+)?x\d+(\.\d+)?', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}wx{extract_number(parts[2])}d"

    # num w x num l or num l x num w -> num h x num w
    elif re.fullmatch(r'\d+(\.\d+)?w[x]\d+(\.\d+)?l', element) or re.fullmatch(r'\d+(\.\d+)?l[x]\d+(\.\d+)?w', element):
        parts = element.split('x')
        return f"{extract_number(parts[1])}hx{extract_number(parts[0])}w"

    # num h x  num l or num l x num h -> num h x num w
    elif re.fullmatch(r'\d+(\.\d+)?h[x]\d+(\.\d+)?l', element) or re.fullmatch(r'\d+(\.\d+)?l[x]\d+(\.\d+)?h', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}w"

    # num h x num l x num w -> num h x num d x num w
    elif re.fullmatch(r'\d+(\.\d+)?h[x]\d+(\.\d+)?l[x]\d+(\.\d+)?w', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}dx{extract_number(parts[2])}w"

    # num w x num l x num d -> num w x num h x num d
    elif re.fullmatch(r'\d+(\.\d+)?w[x]\d+(\.\d+)?l[x]\d+(\.\d+)?d', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}wx{extract_number(parts[1])}hx{extract_number(parts[2])}d"

    # num h x num w x num l -> num h x num w x num d
    elif re.fullmatch(r'\d+(\.\d+)?h[x]\d+(\.\d+)?w[x]\d+(\.\d+)?l', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}wx{extract_number(parts[2])}d"

    # num h x num l x num d -> num h x num w x num d
    elif re.fullmatch(r'\d+(\.\d+)?h[x]\d+(\.\d+)?l[x]\d+(\.\d+)?d', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}wx{extract_number(parts[2])}d"

    # num h x num d x num l -> num h x num d x num w
    elif re.fullmatch(r'\d+(\.\d+)?h[x]\d+(\.\d+)?d[x]\d+(\.\d+)?l', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}hx{extract_number(parts[1])}dx{extract_number(parts[2])}w"

    # num l x num d -> num w x num d
    elif re.fullmatch(r'\d+(\.\d+)?l[x]\d+(\.\d+)?d', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}wx{extract_number(parts[1])}d"

    # num l x num w -> num d x num w
    elif re.fullmatch(r'\d+(\.\d+)?l[x]\d+(\.\d+)?w', element):
        parts = element.split('x')
        return f"{extract_number(parts[0])}dx{extract_number(parts[1])}w"
    else:
        return element

# Format Converted_Size
furniture_df['Converted_Size'] = furniture_df['Converted_Size'].apply(convert_size_format)

# If an element does not include "h", "w", "d", replace with "onesize"
def update_to_onesize(value):
    if not re.search(r'\d+h|\d+w|\d+d', value):
        return "onesize"
    return value

furniture_df['Converted_One_Size'] = furniture_df['Converted_Size'].apply(update_to_onesize)

# Function to extract height, width, depth
def extract_dimension(data, dimension):
    pattern = fr'(\d+\.?\d*){dimension}'  # Regular expression: number followed by "h", "w", or "d"
    extracted = data.str.extract(pattern)  # Extract the number
    return extracted[0].fillna(0).astype(float)  # Convert the result to a float, set missing values to 0

# Create new columns
furniture_df['Height'] = extract_dimension(furniture_df['Converted_Size'], 'h')
furniture_df['Width'] = extract_dimension(furniture_df['Converted_Size'], 'w')
furniture_df['Depth'] = extract_dimension(furniture_df['Converted_Size'], 'd')

# Create a new column for "onesize"
# "onesize" data (One Size, king, queen, etc) are labled as '1', and the rest of the data are labled as '0'
furniture_df['One Size'] = (furniture_df['Converted_One_Size'] == 'onesize').astype(int)

# Create 'measurement units' column

# Function to determine the measurement unit based on Converted_Size
def extract_measurement_unit(element):
    # Check for specific units
    if "sq.ft." in element:
        return 'sq.ft.'
    elif "oz." in element:
        return 'oz.'
    elif "in." in element:
        return 'in.'
    elif "'" in element:  # Check for apostrophe indicating feet
        return 'ft'
    elif element in ["king", "queen", "twin"]:
        return None
    else:
        return 'unknown'  # Default if no matching unit is found

# Function to determine the measurement unit based on 'one size' and 'Converted_Size'
def determine_measurement_unit(row):
    # If 'one size' is 0, force the result to 'in.'
    if row['One Size'] == 0:
        return 'in.'
    # Otherwise, apply the extraction logic
    else:
        return extract_measurement_unit(row['Converted_Size'])

# Apply the function to create a 'measurement units' column
furniture_df['Measurement Units'] = furniture_df.apply(determine_measurement_unit, axis=1)

# Function to assign dimensions for bed sizes adnd create a new column
def assign_dimensions(row):
    if 'king' == row['Converted_Size']:
        return '76" x 80"'
    elif 'queen' == row['Converted_Size']:
        return '60" x 80"'
    elif 'twin' == row['Converted_Size']:
        return '38" x 75"'
    else:
        return None  # If no match, leave as None

# Apply the function to create 'note' column
furniture_df['Notes'] = furniture_df.apply(assign_dimensions, axis=1)

# Delete unnecessary columns
furniture_df = furniture_df.drop(columns=['Converted_Size', 'Converted_One_Size'])


furniture_df

,Product ID / SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,...,Color,Size,Height,Width,Depth,One Size,Measurement Units,Notes,Letters,Numbers
0,W111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,...,Rustic Brown,One Size,0.0,0.0,0.0,1,unknown,None,W,111064236
1,W005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,...,Espresso,"18.2"" H X 41.7"" W X 19.7"" D",18.2,41.7,19.7,0,in.,None,W,005735406
2,W112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,...,White,"14"" H X 25"" L X 21.60"" W",14.0,21.6,25.0,0,in.,None,W,112094006
3,BRWT1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,...,Natural Unfinished,One Size,0.0,0.0,0.0,1,unknown,None,BRWT,1814
4,W100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,...,Gray,"21.6"" X 20.7"" X 20.7""",21.6,20.7,20.7,0,in.,None,W,100762155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Lighting,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,...,Silver,One Size,0.0,0.0,0.0,1,unknown,None,ZHW,1471
1085,EBKN2482,Kitchen Mat,Lighting,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,...,Pink,One Size,0.0,0.0,0.0,1,unknown,None,EBKN,2482
1086,ASKE2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Lighting,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,...,Blue,One Size,0.0,0.0,0.0,1,unknown,None,ASKE,2139
1087,W111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Office,Furniture,Everly Quinn,5299.90,3899.90,NaN,NaN,NaN,...,Blue,One Size,0.0,0.0,0.0,1,unknown,None,W,111971042


# DD Team 4 - SKU<br>
We are separating the letters from the numbers in the SKU column.
The goal is to figure out different meaning behind the letters and/or numbers.

In [48]:
help(str.replace)

Help on method_descriptor:

replace(self, old, new, count=-1, /) unbound builtins.str method
    Return a copy with all occurrences of substring old replaced by new.

      count
        Maximum number of occurrences to replace.
        -1 (the default value) means replace all occurrences.

    If the optional argument count is given, only the first count occurrences are
    replaced.



In [49]:
# DD Team 4
# PRINT LETTERS + NUMBERS of the SKU separated
# pattern to match
pattern = r"([a-zA-Z]+)(\d+)"

# Step 1: Update the Product ID / SKU column with concatenated values
furniture_df['Product ID / SKU'] = furniture_df['Letters'] + '-' + furniture_df['Numbers']

# Step 2: Rename the "Product ID / SKU" column to "SKU"
furniture_df.rename(columns={'Product ID / SKU': 'SKU'}, inplace=True)

# Step 3: Drop the "Letters" and "Numbers" columns
furniture_df.drop(columns=['Letters', 'Numbers'], inplace=True)

# Display the updated dataframe
furniture_df

,SKU,Product Name,Category,Subcategory,Brand,Original Price,Discounted Price,Five Star Reviews,Four Star Reviews,Three Star Reviews,...,Product Size Count,Black Friday Deal,Color,Size,Height,Width,Depth,One Size,Measurement Units,Notes
0,W-111064236,"Jaquavion Coffee Table, Farmhouse Lift Top Cof...",Living Room,Tables,Gracie Oaks,219.99,194.99,19.0,5.0,1.0,...,1,0,Rustic Brown,One Size,0.0,0.0,0.0,1,unknown,None
1,W-005735406,Eoghan Lift Top Coffee Table with 2 Drawers,Living Room,Tables,Millwood Pines,146.99,129.99,2245.0,505.0,182.0,...,2,0,Espresso,"18.2"" H X 41.7"" W X 19.7"" D",18.2,41.7,19.7,0,in.,None
2,W-112094006,Coffee Table,Living Room,Tables,Millwood Pines,689.99,169.99,NaN,NaN,NaN,...,2,0,White,"14"" H X 25"" L X 21.60"" W",14.0,21.6,25.0,0,in.,None
3,BRWT-1814,Esin Single Coffee Table,Living Room,Tables,Breakwater Bay,264.00,134.99,123.0,32.0,10.0,...,1,0,Natural Unfinished,One Size,0.0,0.0,0.0,1,unknown,None
4,W-100762155,Stone Top Coffee Table,Living Room,Tables,Ebern Designs,115.99,115.99,1.0,0.0,0.0,...,2,0,Gray,"21.6"" X 20.7"" X 20.7""",21.6,20.7,20.7,0,in.,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZHW-1471,Stansport Heavy Duty - Stainless Steel Clad Co...,Lighting,Cookware,Stansport,108.99,81.96,5.0,1.0,0.0,...,1,0,Silver,One Size,0.0,0.0,0.0,1,unknown,None
1085,EBKN-2482,Kitchen Mat,Lighting,Rugs,East Urban Home,41.99,41.99,0.0,0.0,1.0,...,1,0,Pink,One Size,0.0,0.0,0.0,1,unknown,None
1086,ASKE-2139,"Vega 12 Piece Stoneware Dinnerware Set, Servic...",Lighting,Accessories,Vancasso,106.99,98.99,NaN,NaN,NaN,...,1,0,Blue,One Size,0.0,0.0,0.0,1,unknown,None
1087,W-111971042,"59.84''Large Size 4-Door Cabinet, Same as Livi...",Office,Furniture,Everly Quinn,5299.90,3899.90,NaN,NaN,NaN,...,1,0,Blue,One Size,0.0,0.0,0.0,1,unknown,None


DD Team 2

In [18]:
# DD Team 2

In [25]:
# Rows to update later
indexes_to_update = [69, 105, 139, 864, 955, 980, 981, 982, 1006, 1007]
indexes_to_add_semicolon = [1172, 1207, 1232]

# Dictionary for typo corrections
typo_corrections = {
    "beige linen blend": "beige linen",
    "brushed nikel": "brushed nickel",
    "charcole": "charcoal gray",
    "charcoal": "charcoal gray",
    "colot": "color",
    "dark dray": "dark gray",
    "gree": "green",
    "geen": "green",
    "glas": "glass",
    "mable": "marble",
    "nikel": "nickel",
    "onecolor": "one color",
    "walnul": "walnut",
}

# Define extended keywords for the 8 main color groups
color_groups = {
    'red': [
        'red', 'scarlet', 'crimson', 'maroon', 'ruby', 'burgundy',
        'cranberry', 'fuchsia', 'rose', 'pink', 'salmon', 'orange', 'cherry'
    ],
    'blue': [
        'blue', 'navy', 'teal', 'cyan', 'azure', 'indigo', 'turquoise', 'aqua',
        'denim', 'ocean', 'lavender', 'purple', 'lilac', 'pale blue', 'sky', 'water color'
    ],
    'green': [
        'green', 'lime', 'olive', 'emerald', 'jade', 'mint', 'seafoam', 'artichoke',
        'fern', 'sage', 'eucalyptus', 'patina', 'camo', 'moss', 'tsucan', 'nature', 'bamboo'
    ],
    'yellow': [
        'yellow', 'gold', 'amber', 'lemon', 'mustard', 'honey', 'sunflower', 'brass',
        'chrome', 'burnished', 'golden'
    ],
    'brown': [
        'brown', 'beige', 'tan', 'walnut', 'chocolate', 'mocha', 'umber', 'almond',
        'bronze', 'camel', 'wood', 'oak', 'maple', 'cedar', 'pine',
        'mahogany', 'biscuit', 'natural', 'praline', 'espresso', 'hazelnut', 'khaki',
        'latte', 'clay', 'teak', 'wheat', 'taupe', 'rust', 'elm',
        'canvas', 'oatmeal', 'pecan', 'castanho', 'walnut', 'rosewood', 'wood', 'copper'
    ],
    'gray': [
        'gray', 'grey', 'charcoal', 'silver', 'ash', 'slate', 'graphite', 'pewter',
        'zinc', 'smoke', 'nickel', 'gunmetal', 'haze', 'greige', 'stainless steel',
        'steel', 'metal', 'aged zinc', 'brushed nickel', 'satin nickel', 'polished nickel',
        'lava gray', 'polished gray', 'polished chrome', 'white washed', 'white washed gray',
        'sharkey gray', 'dark gray', 'light gray', 'cool gray', 'dove gray', 'cloud gray',
        'earthy gray', 'light gun', 'stone gray', 'cement'
    ],
    'white': [
        'white', 'ivory', 'cream', 'snow', 'off-white', 'pearl', 'frost',
        'ice', 'garlic', 'marble', 'crema', 'pure white', 'glossy white',
        'matte white', 'natural white', 'vintage white', 'calacatta white',
        'creamy white', 'white patina', 'white framed', 'one white', 'white with moon and stars'
    ],
    'black': [
        'black', 'jet', 'ebony', 'matte', 'cast iron', 'raven', 'fossil black',
        'antique black', 'satin black', 'mojave matte', 'matte black', 'brushed black',
        'midnight black', 'polished black'
    ]
}

# List of terms to exclude as irrelevant to colors
irrelevant_terms = ['wrapped canvas', 'default', 'transparent']

# Replace colons and commas with semicolons in the "Description" column for specified rows
furniture_df.loc[indexes_to_update, 'Description'] = (
    furniture_df.loc[indexes_to_update, 'Description']
    .str.replace(r'[:,]', ';', regex=True)
)

# Add a semicolon at the beginning of the "Description" column for specified rows
furniture_df.loc[indexes_to_add_semicolon, 'Description'] = (
    furniture_df.loc[indexes_to_add_semicolon, 'Description']
    .apply(lambda x: f";{x}")
)

# Split the "Description" column into two new columns: "extract_color" and "size"
split_df = furniture_df['Description'].str.split(';', n=1, expand=True)

# Assign new column names
split_df.columns = ['extract_color', 'size']

# Add the new columns to the original dataframe
furniture_df = pd.concat([furniture_df, split_df], axis=1)

# Drop the original "Description" column
furniture_df.drop(columns=['Description'], inplace=True)

# Extract the "extract_color" column
color_df = split_df['extract_color']

# Initialize a list to store all colors
all_colors = []

# Process each row in the "extract_color" column
for index, value in color_df.items():
    # Temporarily handle cases with "with moon and stars" to avoid splitting
    if "with moon and stars" in value:
        value = value.replace("with moon and stars", "with moon TEMP_STARS")

    # Perform individual processing
    if isinstance(value, str):
        value = value.replace(" and ", ", ").replace("&", ",").replace("with moon TEMP_STARS", "with moon and stars")

    # Normalize values
    if value in ["N/A", "Null", "Not fill"]:
        value = None

    # Add the value to the list
    all_colors.append(value)

# Replace "/" with "," and split values to generate a preliminary list of colors
all_colors_split = []

for value in all_colors:
    if value is not None:  # Skip None values
        split_colors = [color.strip() for color in value.replace('/', ',').split(',')]
        all_colors_split.extend(split_colors)

# Remove None values, create a unique list of colors, and sort them
unique_colors = sorted(set(color for color in all_colors_split if color is not None))

# Perform preprocessing on colors
cleaned_colors = []
for color in unique_colors:
    # Preprocess colors (e.g., replace "grey" with "gray" and remove extra spaces)
    preprocessed_color = color.lower().strip().replace('grey', 'gray')

    # Correct typos
    cleaned_colors.append(typo_corrections.get(preprocessed_color, preprocessed_color))

# Create a sorted unique list of cleaned colors
unique_cleaned_colors = sorted(set(filter(None, cleaned_colors)))

# Initialize a list to store the mapping table
mapping_table = []

# Process unique colors and cleaned colors simultaneously
for color, cleaned_color in zip(unique_colors, cleaned_colors):
    # Set the initial group as "others"
    grouped_color = 'others'

    # Check each color group
    for group, colors in color_groups.items():
        for item in colors:
            if item in cleaned_color:
                grouped_color = group
                break
        if grouped_color != 'others':
            break

    # Append the original color, cleaned color, and group to the mapping table
    mapping_table.append((color, cleaned_color, grouped_color))

# Create a DataFrame from the mapping table
mapping_df = pd.DataFrame(mapping_table, columns=["Extract color", "Cleaned Color", "Grouped Color"])

# Create a unique list of cleaned colors
cleaned_color_list = mapping_df['Cleaned Color'].drop_duplicates().tolist()

# Initialize a new table with necessary columns
new_table = furniture_df[['Product ID / SKU', 'extract_color']].copy()

# Add columns for cleaned colors, initialized with zeros
cleaned_color_columns = pd.DataFrame(0, index=new_table.index, columns=[str(cleaned_color) for cleaned_color in cleaned_color_list])
new_table = pd.concat([new_table, cleaned_color_columns], axis=1)

# Logic to set flags for matching colors
for index, row in new_table.iterrows():
    extract_color = row['extract_color']

    if isinstance(extract_color, str):
        # Handle special cases and split extract_color
        if extract_color == "Gray and white with moon and stars/blue and white with moon and stars/pink and white with moon and stars/white":
            parts = extract_color.split('/')
            extract_colors_list = []
            for part in parts:
                sub_parts = re.split(r'\s+and\s+', part)
                combined_parts = []
                for sub_part in sub_parts:
                    if "with moon and stars" in sub_part:
                        combined_parts.append(sub_part.strip().lower())
                    else:
                        combined_parts.extend(sub_part.strip().lower().split())
                extract_colors_list.extend(combined_parts)
        else:
            extract_colors_list = [color.strip().lower() for color in re.split(r'[,/]', extract_color)]

        # Check matches for each color
        for _, def_row in mapping_df.iterrows():
            original_color = def_row['Extract color'].strip().lower()
            cleaned_color = def_row['Cleaned Color']

            if original_color in extract_colors_list:
                new_table.at[index, cleaned_color] = 1

# Create headers for the detailed color table
header_row_1 = [''] * 2
for cleaned_color in cleaned_color_list:
    grouped_color = mapping_df.loc[mapping_df['Cleaned Color'] == cleaned_color, 'Grouped Color']
    header_row_1.append(grouped_color.iloc[0] if not grouped_color.empty else '')

header_row_2 = ['Product ID / SKU', 'Extract color'] + cleaned_color_list

# Adjust the number of headers
while len(header_row_1) < len(new_table.columns):
    header_row_1.append('')

# Combine headers and data
header_df = pd.DataFrame([header_row_1, header_row_2], columns=new_table.columns)
final_table = pd.concat([header_df, new_table], ignore_index=True)

# Display the final table
print("\nDetailed Color Table:")
final_table

KeyError: 'Description'

DD Team 3

In [ ]:
# DD Team 3